# Imports

In [109]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
import time
import sys
import random

#### Set K

In [227]:
k = 2

grid_size = 2*k + 1
search_size = 2

### Path to original video file

In [2]:
path_to_video = './monkey/monkey.avi'
frame_save_path = './frames/'

### Helper functions

In [ ]:
def arrowdraw(img, x1, y1, x2, y2, ssd):
    radians = math.atan2(x1-x2, y2-y1)
    x11 = 0
    y11 = 0
    x12 = -10
    y12 = -10

    u11 = 0
    v11 = 0
    u12 = 10
    v12 = -10
    
    x11_ = x11*math.cos(radians) - y11*math.sin(radians) + x2
    y11_ = x11*math.sin(radians) + y11*math.cos(radians) + y2

    x12_ = x12 * math.cos(radians) - y12 * math.sin(radians) + x2
    y12_ = x12 * math.sin(radians) + y12 * math.cos(radians) + y2
    
    u11_ = u11 * math.cos(radians) - v11 * math.sin(radians) + x2
    v11_ = u11 * math.sin(radians) + v11 * math.cos(radians) + y2

    u12_ = u12 * math.cos(radians) - v12 * math.sin(radians) + x2
    v12_ = u12 * math.sin(radians) + v12 * math.cos(radians) + y2
    img = cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 2)
    img = cv2.line(img, (int(x11_), int(y11_)), (int(x12_), int(y12_)), 
    (255, 255, 255), 1)
    img = cv2.line(img, (int(u11_), int(v11_)), (int(u12_), int(v12_)), 
    (255, 255, 255), 1)
    # cv2.putText(img=img, text=f'{ssd}', org=(x1, y1)
    #             , fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=0.4,
    #             color=(0, 0, 0), thickness=1)
    
    return img

def draw_box(img, top_left, bottom_left, top_right, bottom_right):
    R = random.randint(0, 255)
    G = random.randint(0, 255)
    B = random.randint(0, 255)
    img = cv2.line(img, top_left, top_right, (R, G, B), 5)
    img = cv2.line(img, top_left, bottom_left, (R, G, B), 5)
    img = cv2.line(img, bottom_right, bottom_left, (R, G, B), 5)
    img = cv2.line(img, bottom_right, top_right, (R, G, B), 5)

    return img

def sqrt_ssd_function(current_frame, next_frame, top_left_of_grid, current_frame_num):
    BLUE = 120
    min_ssd = 999999999999
    top_left_of_most_similar_block = (-1, -1)
    for search_x in range(-search_size * grid_size + top_left_of_grid[0], search_size * grid_size + top_left_of_grid[0], grid_size):
        for search_y in range(-search_size * grid_size + top_left_of_grid[1], search_size * grid_size + top_left_of_grid[1], grid_size):
            if (
                search_x < 0 or
                search_y < 0 or
                search_x + grid_size >= current_frame.shape[1] or
                search_y + grid_size >= current_frame.shape[0] 
            ):
                continue
            # colourR = random.randint(0, 255)
            # colourG = random.randint(0, 255)
            # colourB = random.randint(0, 255)
            # top_left_corner = (search_x, search_y)
            # bottom_left_corner = (search_x, search_y + grid_size)
            # top_right_corner = (search_x+grid_size, search_y)
            # bottom_right_corner = (search_x + grid_size, search_y + grid_size)
            
            # current_frame = draw_box(current_frame, top_left_corner, bottom_left_corner, top_right_corner, bottom_right_corner, colourR, colourG, colourB)
            # cv2.imwrite(f'debug/grid{top_left_of_grid}x{search_x}y{search_y}frame{current_frame_num}.tif', current_frame)
            sum_under_sqrt = 0
            for x in range(search_x, search_x + grid_size):
                for y in range(search_y, search_y + grid_size):
                    # relative_x_in_current_frame = x - search_x
                    # relative_y_in_current_frame = y - search_y
                    # if current_frame[y][x][0] >= BLUE:
                    #     continue
                    for colour in range(3):
                        sum_under_sqrt += math.pow(int(current_frame[y][x][colour]) - int(next_frame[y][x][colour]), 2)
            
            sum_of_squared_distances = math.sqrt(sum_under_sqrt)
            if sum_of_squared_distances < min_ssd:
                min_ssd = sum_of_squared_distances
                top_left_of_most_similar_block = (search_x, search_y)

            
    return top_left_of_most_similar_block, min_ssd

### Extract frames from original video file

Using code from previous labs to do this

In [188]:
cap = cv2.VideoCapture(path_to_video)
if not cap.isOpened():
    print('{} not opened'.format(path_to_video))
    sys.exit(1)
time_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_counter = 0                                             # FRAME_COUNTER
while(1):
    return_flag, frame = cap.read()  
    if not return_flag:
        print('Video Reach End')
        break
    # Main Content - Start
    cv2.imshow('Monkey', frame)
    cv2.imwrite(frame_save_path + 'frame%d.tif' % frame_counter, frame)
    frame_counter += 1
    # Main Content - End    
    if cv2.waitKey(30) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Video Reach End


### Main Body

In [228]:
current_frame = 0
# counter = 0
while(1):

    img = cv2.imread('frames/frame%d.tif' % current_frame)
    next_img = cv2.imread('frames/frame%d.tif' % (current_frame + 1))
    if next_img is None:
        print('ooops! no bg found frames/frame%d.tif' % (current_frame + 1))
        break
    
    displacement_vectors = []

    for grid_block_y in range(0, img.shape[0], grid_size):
        for grid_block_x in range(0, img.shape[1], grid_size):
            if grid_block_x + grid_size > img.shape[1] or grid_block_y + grid_size > img.shape[0]:
                continue
            centroid_x = grid_block_x + math.ceil(grid_size/2)
            centroid_y = grid_block_y + math.ceil(grid_size/2)
            top_left_corner = (grid_block_x, grid_block_y)

            top_left_corner_of_most_similar_block, ssd = sqrt_ssd_function(img, next_img, top_left_corner, current_frame)
            if ssd < 125 or ssd > 200:
                continue
            to_centroid_x = top_left_corner_of_most_similar_block[0] + math.ceil(grid_size/2)
            to_centroid_y = top_left_corner_of_most_similar_block[1] + math.ceil(grid_size/2)
            displacement_vectors.append((centroid_x, centroid_y, to_centroid_x, to_centroid_y, int(ssd)))

            # bottom_left_corner = (grid_block_x, grid_block_y + grid_size)
            # top_right_corner = (grid_block_x+grid_size, grid_block_y)
            # bottom_right_corner = (grid_block_x + grid_size, grid_block_y + grid_size)
            # print(top_left_corner_of_most_similar_block)
            
            # img = draw_box(img, top_left_corner, bottom_left_corner, top_right_corner, bottom_right_corner)
        # counter += 1
        # cv2.imwrite('processed/frame%d.tif' % grid_block_y, img)

    # print(displacement_vectors)
    already_destination = {}
    for vector in displacement_vectors:
        if (vector[2], vector[3]) in already_destination.keys():
            continue
        already_destination[(vector[2], vector[3])] = 1
        arrowdraw(img, vector[0], vector[1], vector[2], vector[3], vector[4])

    cv2.imwrite('processed/frame%d.tif' % current_frame, img)

    current_frame += 1

[(198, 3, 188, 8, 65), (198, 8, 188, 8, 65), (198, 13, 188, 18, 60), (198, 18, 188, 23, 40), (198, 23, 188, 23, 40), (198, 28, 188, 23, 40), (198, 33, 188, 38, 33), (203, 38, 208, 33, 31), (203, 43, 208, 33, 31), (203, 48, 208, 38, 39), (203, 53, 208, 43, 48), (198, 58, 188, 53, 30), (203, 58, 193, 58, 82), (203, 63, 193, 68, 81), (203, 68, 193, 73, 64), (203, 73, 193, 78, 38), (203, 78, 193, 78, 38), (203, 83, 193, 88, 32), (208, 98, 213, 88, 37), (208, 103, 213, 103, 40), (208, 108, 213, 103, 40), (208, 113, 213, 103, 40), (208, 118, 198, 123, 40), (213, 158, 218, 153, 32), (213, 163, 218, 153, 32), (213, 168, 203, 168, 47), (213, 173, 203, 168, 47), (213, 178, 203, 168, 47), (213, 183, 203, 188, 41), (213, 188, 203, 193, 30), (303, 268, 293, 258, 43), (308, 268, 298, 258, 61), (313, 268, 303, 258, 74), (318, 268, 308, 258, 83), (323, 268, 328, 258, 64), (328, 268, 333, 258, 38), (293, 273, 283, 263, 34), (298, 273, 288, 263, 56), (303, 273, 293, 263, 133), (308, 273, 298, 263, 230),

KeyboardInterrupt: 

### Convert the frames to a video

In [210]:
frame_load_path = './processed/'
path_to_output_video = './predictableMonkey.mp4'

out = cv2.VideoWriter(path_to_output_video, cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 24, (int(frame_width), int(frame_height)))
frame_counter = 0
while(1):
    img = cv2.imread(frame_load_path + 'frame%d.tif' % frame_counter)
    if img is None:
        print('No more frames to be loaded')
        break
    out.write(img)
    frame_counter += 1
out.release()
cv2.destroyAllWindows()

No more frames to be loaded


[ WARN:0@99731.298] global loadsave.cpp:248 findDecoder imread_('./processed/frame751.tif'): can't open/read file: check file path/integrity
